In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

In [3]:
from optimus import optimus as Optimus

C:\Users\argenisleon\Anaconda3\lib\site-packages\socks.py:58: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Callable
C:\Users\argenisleon\Anaconda3\lib\site-packages\statsmodels\base\wrapper.py:100: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  defaults=argspec[3])

    You are using PySparkling of version 2.4.10, but your PySpark is of
    version 2.3.1. Please make sure Spark and PySparkling versions are compatible. 


In [4]:
op = Optimus("dask")

In [5]:
df = op.load.csv("data/crime.csv")

In [7]:
df.cols.min("*")

invalid escape sequence \d


{'INCIDENT_NUMBER': {'min': '142052550'},
 'OFFENSE_CODE': {'min': 111},
 'OFFENSE_CODE_GROUP': {'min': 'Aggravated Assault'},
 'OFFENSE_DESCRIPTION': {'min': 'A&B HANDS, FEET, ETC.  - MED. ATTENTION REQ.'},
 'REPORTING_AREA': {'min': ' '},
 'OCCURRED_ON_DATE': {'min': '2015-06-15 00:00:00'},
 'YEAR': {'min': 2015},
 'MONTH': {'min': 1},
 'DAY_OF_WEEK': {'min': 'Friday'},
 'HOUR': {'min': 0},
 'Lat': {'min': -1.0},
 'Long': {'min': -71.17867378},
 'Location': {'min': '(-1.00000000, -1.00000000)'}}

In [8]:
df.cols.max("*")

{'INCIDENT_NUMBER': {'max': 'I182070945'},
 'OFFENSE_CODE': {'max': 3831},
 'OFFENSE_CODE_GROUP': {'max': 'Warrant Arrests'},
 'OFFENSE_DESCRIPTION': {'max': 'WEAPON - OTHER - OTHER VIOLATION'},
 'REPORTING_AREA': {'max': '99'},
 'OCCURRED_ON_DATE': {'max': '2018-09-03 21:25:00'},
 'YEAR': {'max': 2018},
 'MONTH': {'max': 12},
 'DAY_OF_WEEK': {'max': 'Wednesday'},
 'HOUR': {'max': 23},
 'Lat': {'max': 42.39504158},
 'Long': {'max': -1.0},
 'Location': {'max': '(42.39504158, -71.01017732)'}}

In [60]:
a = df[["OFFENSE_CODE"]].quantile([0.5])
for k,v in a.iteritems():
    print(i)

AttributeError: 'DataFrame' object has no attribute 'iteritems'

In [36]:
df[["OFFENSE_CODE"]].min()

Dask Series Structure:
npartitions=1
OFFENSE_CODE    int64
OFFENSE_CODE      ...
dtype: int64
Dask Name: dataframe-min-agg, 6 tasks

In [64]:
df.cols.percentile("*", values=[0.5,0.95])

{'OFFENSE_CODE': {'percentile': 0.50    2907.0
  0.95    3831.0
  Name: OFFENSE_CODE, dtype: float64}, 'YEAR': {'percentile': 0.50    2017.0
  0.95    2018.0
  Name: YEAR, dtype: float64}, 'MONTH': {'percentile': 0.50     7.0
  0.95    12.0
  Name: MONTH, dtype: float64}, 'HOUR': {'percentile': 0.50    14.0
  0.95    22.0
  Name: HOUR, dtype: float64}, 'Lat': {'percentile': 0.50    42.325538
  0.95    42.372159
  Name: Lat, dtype: float64}, 'Long': {'percentile': 0.50   -71.077524
  0.95   -71.039799
  Name: Long, dtype: float64}}

In [62]:
# df.cols.percentile("*")
{"percentile": {str(i): j for i, j in df.quantile(0.5).iteritems()}}

{'percentile': {'OFFENSE_CODE': 2907.0,
  'YEAR': 2017.0,
  'MONTH': 7.0,
  'HOUR': 14.0,
  'Lat': 42.32553801,
  'Long': -71.07752376}}

In [7]:
# %reset_selective?

In [23]:
for i in df.cols.names():
    df.cols.kurtosis(i)

AttributeError: 'Cols' object has no attribute 'kurtosis'

In [105]:
df.cols.min("*")

{'INCIDENT_NUMBER': {'min': '142052550'},
 'OFFENSE_CODE': {'min': 111},
 'OFFENSE_CODE_GROUP': {'min': 'Aggravated Assault'},
 'OFFENSE_DESCRIPTION': {'min': 'A&B HANDS, FEET, ETC.  - MED. ATTENTION REQ.'},
 'REPORTING_AREA': {'min': ' '},
 'OCCURRED_ON_DATE': {'min': '2015-06-15 00:00:00'},
 'YEAR': {'min': 2015},
 'MONTH': {'min': 1},
 'DAY_OF_WEEK': {'min': 'Friday'},
 'HOUR': {'min': 0},
 'Lat': {'min': -1.0},
 'Long': {'min': -71.17867378},
 'Location': {'min': '(-1.00000000, -1.00000000)'}}

In [41]:
%%time
a = df.min().compute()
print(type(a))
# print({k:{"min":v} for k,v in a.items()}    )

<class 'pandas.core.series.Series'>
Wall time: 2.53 s


In [9]:
df.cols.lower("INCIDENT_NUMBER").ext.display()

INCIDENT_NUMBER INCIDENT_NUMBER None <class 'str'>


INCIDENT_NUMBER 1 (object) not nullable,OFFENSE_CODE 2 (int64) not nullable,OFFENSE_CODE_GROUP 3 (object) not nullable,OFFENSE_DESCRIPTION 4 (object) not nullable,DISTRICT 5 (object) not nullable,REPORTING_AREA 6 (object) not nullable,SHOOTING 7 (object) not nullable,OCCURRED_ON_DATE 8 (object) not nullable,YEAR 9 (int64) not nullable,MONTH 10 (int64) not nullable,DAY_OF_WEEK 11 (object) not nullable,HOUR 12 (int64) not nullable,UCR_PART 13 (object) not nullable,STREET 14 (object) not nullable,Lat 15 (float64) not nullable,Long 16 (float64) not nullable,Location 17 (object) not nullable
i182070945,619,Larceny,LARCENY⋅ALL⋅OTHERS,D14,808,nan,2018-09-02⋅13:00:00,2018,9,Sunday,13,Part⋅One,LINCOLN⋅ST,42.35779134,-71.13937053,"(42.35779134,⋅-71.13937053)"
i182070943,1402,Vandalism,VANDALISM,C11,347,nan,2018-08-21⋅00:00:00,2018,8,Tuesday,0,Part⋅Two,HECLA⋅ST,42.30682138,-71.06030035,"(42.30682138,⋅-71.06030035)"
i182070941,3410,Towed,TOWED⋅MOTOR⋅VEHICLE,D4,151,nan,2018-09-03⋅19:27:00,2018,9,Monday,19,Part⋅Three,CAZENOVE⋅ST,42.34658879,-71.07242943,"(42.34658879,⋅-71.07242943)"
i182070940,3114,Investigate⋅Property,INVESTIGATE⋅PROPERTY,D4,272,nan,2018-09-03⋅21:16:00,2018,9,Monday,21,Part⋅Three,NEWCOMB⋅ST,42.33418175,-71.07866441,"(42.33418175,⋅-71.07866441)"
i182070938,3114,Investigate⋅Property,INVESTIGATE⋅PROPERTY,B3,421,nan,2018-09-03⋅21:05:00,2018,9,Monday,21,Part⋅Three,DELHI⋅ST,42.27536542,-71.09036101,"(42.27536542,⋅-71.09036101)"
i182070936,3820,Motor⋅Vehicle⋅Accident⋅Response,M/V⋅ACCIDENT⋅INVOLVING⋅PEDESTRIAN⋅-⋅INJURY,C11,398,nan,2018-09-03⋅21:09:00,2018,9,Monday,21,Part⋅Three,TALBOT⋅AVE,42.29019621,-71.07159012,"(42.29019621,⋅-71.07159012)"
i182070933,724,Auto⋅Theft,AUTO⋅THEFT,B2,330,nan,2018-09-03⋅21:25:00,2018,9,Monday,21,Part⋅One,NORMANDY⋅ST,42.30607218,-71.0827326,"(42.30607218,⋅-71.08273260)"
i182070932,3301,Verbal⋅Disputes,VERBAL⋅DISPUTE,B2,584,nan,2018-09-03⋅20:39:37,2018,9,Monday,20,Part⋅Three,LAWN⋅ST,42.32701648,-71.10555088,"(42.32701648,⋅-71.10555088)"
i182070931,301,Robbery,ROBBERY⋅-⋅STREET,C6,177,nan,2018-09-03⋅20:48:00,2018,9,Monday,20,Part⋅One,MASSACHUSETTS⋅AVE,42.33152148,-71.07085307,"(42.33152148,⋅-71.07085307)"
i182070929,3301,Verbal⋅Disputes,VERBAL⋅DISPUTE,C11,364,nan,2018-09-03⋅20:38:00,2018,9,Monday,20,Part⋅Three,LESLIE⋅ST,42.29514664,-71.05860832,"(42.29514664,⋅-71.05860832)"


In [13]:
df.cols.upper("OFFENSE_CODE_GROUP").ext.display()

OFFENSE_CODE_GROUP OFFENSE_CODE_GROUP None <class 'str'>


INCIDENT_NUMBER 1 (object) not nullable,OFFENSE_CODE 2 (int64) not nullable,OFFENSE_CODE_GROUP 3 (object) not nullable,OFFENSE_DESCRIPTION 4 (object) not nullable,DISTRICT 5 (object) not nullable,REPORTING_AREA 6 (object) not nullable,SHOOTING 7 (object) not nullable,OCCURRED_ON_DATE 8 (object) not nullable,YEAR 9 (int64) not nullable,MONTH 10 (int64) not nullable,DAY_OF_WEEK 11 (object) not nullable,HOUR 12 (int64) not nullable,UCR_PART 13 (object) not nullable,STREET 14 (object) not nullable,Lat 15 (float64) not nullable,Long 16 (float64) not nullable,Location 17 (object) not nullable
I182070945,619,LARCENY,LARCENY⋅ALL⋅OTHERS,D14,808,nan,2018-09-02⋅13:00:00,2018,9,Sunday,13,Part⋅One,LINCOLN⋅ST,42.35779134,-71.13937053,"(42.35779134,⋅-71.13937053)"
I182070943,1402,VANDALISM,VANDALISM,C11,347,nan,2018-08-21⋅00:00:00,2018,8,Tuesday,0,Part⋅Two,HECLA⋅ST,42.30682138,-71.06030035,"(42.30682138,⋅-71.06030035)"
I182070941,3410,TOWED,TOWED⋅MOTOR⋅VEHICLE,D4,151,nan,2018-09-03⋅19:27:00,2018,9,Monday,19,Part⋅Three,CAZENOVE⋅ST,42.34658879,-71.07242943,"(42.34658879,⋅-71.07242943)"
I182070940,3114,INVESTIGATE⋅PROPERTY,INVESTIGATE⋅PROPERTY,D4,272,nan,2018-09-03⋅21:16:00,2018,9,Monday,21,Part⋅Three,NEWCOMB⋅ST,42.33418175,-71.07866441,"(42.33418175,⋅-71.07866441)"
I182070938,3114,INVESTIGATE⋅PROPERTY,INVESTIGATE⋅PROPERTY,B3,421,nan,2018-09-03⋅21:05:00,2018,9,Monday,21,Part⋅Three,DELHI⋅ST,42.27536542,-71.09036101,"(42.27536542,⋅-71.09036101)"
I182070936,3820,MOTOR⋅VEHICLE⋅ACCIDENT⋅RESPONSE,M/V⋅ACCIDENT⋅INVOLVING⋅PEDESTRIAN⋅-⋅INJURY,C11,398,nan,2018-09-03⋅21:09:00,2018,9,Monday,21,Part⋅Three,TALBOT⋅AVE,42.29019621,-71.07159012,"(42.29019621,⋅-71.07159012)"
I182070933,724,AUTO⋅THEFT,AUTO⋅THEFT,B2,330,nan,2018-09-03⋅21:25:00,2018,9,Monday,21,Part⋅One,NORMANDY⋅ST,42.30607218,-71.0827326,"(42.30607218,⋅-71.08273260)"
I182070932,3301,VERBAL⋅DISPUTES,VERBAL⋅DISPUTE,B2,584,nan,2018-09-03⋅20:39:37,2018,9,Monday,20,Part⋅Three,LAWN⋅ST,42.32701648,-71.10555088,"(42.32701648,⋅-71.10555088)"
I182070931,301,ROBBERY,ROBBERY⋅-⋅STREET,C6,177,nan,2018-09-03⋅20:48:00,2018,9,Monday,20,Part⋅One,MASSACHUSETTS⋅AVE,42.33152148,-71.07085307,"(42.33152148,⋅-71.07085307)"
I182070929,3301,VERBAL⋅DISPUTES,VERBAL⋅DISPUTE,C11,364,nan,2018-09-03⋅20:38:00,2018,9,Monday,20,Part⋅Three,LESLIE⋅ST,42.29514664,-71.05860832,"(42.29514664,⋅-71.05860832)"


In [26]:
df.str.upper()

AttributeError: 'DataFrame' object has no attribute 'str'

In [24]:
df.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,LAT,LONG,LOCATION
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


In [16]:
df.cols.trim("OFFENSE_CODE_GROUP").ext.display()

OFFENSE_CODE_GROUP OFFENSE_CODE_GROUP None <class 'str'>


INCIDENT_NUMBER 1 (object) not nullable,OFFENSE_CODE 2 (int64) not nullable,OFFENSE_CODE_GROUP 3 (object) not nullable,OFFENSE_DESCRIPTION 4 (object) not nullable,DISTRICT 5 (object) not nullable,REPORTING_AREA 6 (object) not nullable,SHOOTING 7 (object) not nullable,OCCURRED_ON_DATE 8 (object) not nullable,YEAR 9 (int64) not nullable,MONTH 10 (int64) not nullable,DAY_OF_WEEK 11 (object) not nullable,HOUR 12 (int64) not nullable,UCR_PART 13 (object) not nullable,STREET 14 (object) not nullable,Lat 15 (float64) not nullable,Long 16 (float64) not nullable,Location 17 (object) not nullable
I182070945,619,Larceny,LARCENY⋅ALL⋅OTHERS,D14,808,nan,2018-09-02⋅13:00:00,2018,9,Sunday,13,Part⋅One,LINCOLN⋅ST,42.35779134,-71.13937053,"(42.35779134,⋅-71.13937053)"
I182070943,1402,Vandalism,VANDALISM,C11,347,nan,2018-08-21⋅00:00:00,2018,8,Tuesday,0,Part⋅Two,HECLA⋅ST,42.30682138,-71.06030035,"(42.30682138,⋅-71.06030035)"
I182070941,3410,Towed,TOWED⋅MOTOR⋅VEHICLE,D4,151,nan,2018-09-03⋅19:27:00,2018,9,Monday,19,Part⋅Three,CAZENOVE⋅ST,42.34658879,-71.07242943,"(42.34658879,⋅-71.07242943)"
I182070940,3114,Investigate⋅Property,INVESTIGATE⋅PROPERTY,D4,272,nan,2018-09-03⋅21:16:00,2018,9,Monday,21,Part⋅Three,NEWCOMB⋅ST,42.33418175,-71.07866441,"(42.33418175,⋅-71.07866441)"
I182070938,3114,Investigate⋅Property,INVESTIGATE⋅PROPERTY,B3,421,nan,2018-09-03⋅21:05:00,2018,9,Monday,21,Part⋅Three,DELHI⋅ST,42.27536542,-71.09036101,"(42.27536542,⋅-71.09036101)"
I182070936,3820,Motor⋅Vehicle⋅Accident⋅Response,M/V⋅ACCIDENT⋅INVOLVING⋅PEDESTRIAN⋅-⋅INJURY,C11,398,nan,2018-09-03⋅21:09:00,2018,9,Monday,21,Part⋅Three,TALBOT⋅AVE,42.29019621,-71.07159012,"(42.29019621,⋅-71.07159012)"
I182070933,724,Auto⋅Theft,AUTO⋅THEFT,B2,330,nan,2018-09-03⋅21:25:00,2018,9,Monday,21,Part⋅One,NORMANDY⋅ST,42.30607218,-71.0827326,"(42.30607218,⋅-71.08273260)"
I182070932,3301,Verbal⋅Disputes,VERBAL⋅DISPUTE,B2,584,nan,2018-09-03⋅20:39:37,2018,9,Monday,20,Part⋅Three,LAWN⋅ST,42.32701648,-71.10555088,"(42.32701648,⋅-71.10555088)"
I182070931,301,Robbery,ROBBERY⋅-⋅STREET,C6,177,nan,2018-09-03⋅20:48:00,2018,9,Monday,20,Part⋅One,MASSACHUSETTS⋅AVE,42.33152148,-71.07085307,"(42.33152148,⋅-71.07085307)"
I182070929,3301,Verbal⋅Disputes,VERBAL⋅DISPUTE,C11,364,nan,2018-09-03⋅20:38:00,2018,9,Monday,20,Part⋅Three,LESLIE⋅ST,42.29514664,-71.05860832,"(42.29514664,⋅-71.05860832)"


In [29]:
# def polar(data):
#     data=scale(sid.polarity_scores(data.tweet)['compound'])
#     return data

# hola_df.map_partitions(clean_text, meta=df)
def _lower(text, args):
    a = args[0]
    return text[a].str.lower()

def apply(df,cols, func):
    kwargs = {cols:df[[cols]].map_partitions(_lower, args=(cols,1), meta=(cols,str))}
    return df.assign(**kwargs)
    

apply(df, "INCIDENT_NUMBER", _lower).head()
# df["INCIDENT_NUMBER"].map_partitions(_lower, meta=("INCIDENT_NUMBER",str)).head()


,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,LAT,LONG,LOCATION
0,i182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,i182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,i182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,i182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,i182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


In [49]:
df1["hola"] = hola_df["INCIDENT_NUMBER"].replace(20,0)

AttributeError: 'Series' object has no attribute 'replace'

In [9]:
df1 =  df.cols.replace("*",["a","b","c"],"")

AttributeError: 'Cols' object has no attribute 'replace'

In [48]:
df1.ext.display()

INCIDENT_NUMBER 1 (string) nullable,OFFENSE_CODE 2 (string) nullable,OFFENSE_CODE_GROUP 3 (string) nullable,OFFENSE_DESCRIPTION 4 (string) nullable,DISTRICT 5 (string) nullable,REPORTING_AREA 6 (string) nullable,SHOOTING 7 (string) nullable,OCCURRED_ON_DATE 8 (timestamp) nullable,YEAR 9 (string) nullable,MONTH 10 (string) nullable,DAY_OF_WEEK 11 (string) nullable,HOUR 12 (string) nullable,UCR_PART 13 (string) nullable,STREET 14 (string) nullable,Lat 15 (string) nullable,Long 16 (string) nullable,Location 17 (string) nullable
I182070945,619,Lreny,LARCENY⋅ALL⋅OTHERS,D14,808,None,2018-09-02⋅13:00:00,2018,9,Sundy,13,Prt⋅One,LINCOLN⋅ST,42.35779134,-71.13937053,"(42.35779134,⋅-71.13937053)"
I182070943,1402,Vndlism,VANDALISM,C11,347,None,2018-08-21⋅00:00:00,2018,8,Tuesdy,0,Prt⋅Two,HECLA⋅ST,42.30682138,-71.06030035,"(42.30682138,⋅-71.06030035)"
I182070941,3410,Towed,TOWED⋅MOTOR⋅VEHICLE,D4,151,None,2018-09-03⋅19:27:00,2018,9,Mondy,19,Prt⋅Three,CAZENOVE⋅ST,42.34658879,-71.07242943,"(42.34658879,⋅-71.07242943)"
I182070940,3114,Investigte⋅Property,INVESTIGATE⋅PROPERTY,D4,272,None,2018-09-03⋅21:16:00,2018,9,Mondy,21,Prt⋅Three,NEWCOMB⋅ST,42.33418175,-71.07866441,"(42.33418175,⋅-71.07866441)"
I182070938,3114,Investigte⋅Property,INVESTIGATE⋅PROPERTY,B3,421,None,2018-09-03⋅21:05:00,2018,9,Mondy,21,Prt⋅Three,DELHI⋅ST,42.27536542,-71.09036101,"(42.27536542,⋅-71.09036101)"
I182070936,3820,Motor⋅Vehile⋅Aident⋅Response,M/V⋅ACCIDENT⋅INVOLVING⋅PEDESTRIAN⋅-⋅INJURY,C11,398,None,2018-09-03⋅21:09:00,2018,9,Mondy,21,Prt⋅Three,TALBOT⋅AVE,42.29019621,-71.07159012,"(42.29019621,⋅-71.07159012)"
I182070933,724,Auto⋅Theft,AUTO⋅THEFT,B2,330,None,2018-09-03⋅21:25:00,2018,9,Mondy,21,Prt⋅One,NORMANDY⋅ST,42.30607218,-71.0827326,"(42.30607218,⋅-71.08273260)"
I182070932,3301,Verl⋅Disputes,VERBAL⋅DISPUTE,B2,584,None,2018-09-03⋅20:39:37,2018,9,Mondy,20,Prt⋅Three,LAWN⋅ST,42.32701648,-71.10555088,"(42.32701648,⋅-71.10555088)"
I182070931,301,Roery,ROBBERY⋅-⋅STREET,C6,177,None,2018-09-03⋅20:48:00,2018,9,Mondy,20,Prt⋅One,MASSACHUSETTS⋅AVE,42.33152148,-71.07085307,"(42.33152148,⋅-71.07085307)"
I182070929,3301,Verl⋅Disputes,VERBAL⋅DISPUTE,C11,364,None,2018-09-03⋅20:38:00,2018,9,Mondy,20,Prt⋅Three,LESLIE⋅ST,42.29514664,-71.05860832,"(42.29514664,⋅-71.05860832)"


In [7]:
df.cols.count_by_dtypes("OFFENSE_CODE")

{'OFFENSE_CODE': {'int': 319073, 'float': 0}}

In [7]:
df.cols.percentile("OFFENSE_CODE",[0.5])

invalid escape sequence \d


[('OFFENSE_CODE', {'percentile': {'0.5': 2907.0}})]


{'OFFENSE_CODE': {'percentile': {'0.5': 2907.0}}}

In [8]:
df.cols.median("*")

TypeError: can't multiply sequence by non-int of type 'float'

In [28]:
df.cols.hist("OFFENSE_CODE")

AttributeError: 'Cols' object has no attribute 'hist'

In [10]:
df.ext.display()

INCIDENT_NUMBER 1 (object) not nullable,OFFENSE_CODE 2 (int64) not nullable,OFFENSE_CODE_GROUP 3 (object) not nullable,OFFENSE_DESCRIPTION 4 (object) not nullable,DISTRICT 5 (object) not nullable,REPORTING_AREA 6 (object) not nullable,SHOOTING 7 (object) not nullable,OCCURRED_ON_DATE 8 (object) not nullable,YEAR 9 (int64) not nullable,MONTH 10 (int64) not nullable,DAY_OF_WEEK 11 (object) not nullable,HOUR 12 (int64) not nullable,UCR_PART 13 (object) not nullable,STREET 14 (object) not nullable,Lat 15 (float64) not nullable,Long 16 (float64) not nullable,Location 17 (object) not nullable
I182070945,619,Larceny,LARCENY⋅ALL⋅OTHERS,D14,808,nan,2018-09-02⋅13:00:00,2018,9,Sunday,13,Part⋅One,LINCOLN⋅ST,42.35779134,-71.13937053,"(42.35779134,⋅-71.13937053)"
I182070943,1402,Vandalism,VANDALISM,C11,347,nan,2018-08-21⋅00:00:00,2018,8,Tuesday,0,Part⋅Two,HECLA⋅ST,42.30682138,-71.06030035,"(42.30682138,⋅-71.06030035)"
I182070941,3410,Towed,TOWED⋅MOTOR⋅VEHICLE,D4,151,nan,2018-09-03⋅19:27:00,2018,9,Monday,19,Part⋅Three,CAZENOVE⋅ST,42.34658879,-71.07242943,"(42.34658879,⋅-71.07242943)"
I182070940,3114,Investigate⋅Property,INVESTIGATE⋅PROPERTY,D4,272,nan,2018-09-03⋅21:16:00,2018,9,Monday,21,Part⋅Three,NEWCOMB⋅ST,42.33418175,-71.07866441,"(42.33418175,⋅-71.07866441)"
I182070938,3114,Investigate⋅Property,INVESTIGATE⋅PROPERTY,B3,421,nan,2018-09-03⋅21:05:00,2018,9,Monday,21,Part⋅Three,DELHI⋅ST,42.27536542,-71.09036101,"(42.27536542,⋅-71.09036101)"
I182070936,3820,Motor⋅Vehicle⋅Accident⋅Response,M/V⋅ACCIDENT⋅INVOLVING⋅PEDESTRIAN⋅-⋅INJURY,C11,398,nan,2018-09-03⋅21:09:00,2018,9,Monday,21,Part⋅Three,TALBOT⋅AVE,42.29019621,-71.07159012,"(42.29019621,⋅-71.07159012)"
I182070933,724,Auto⋅Theft,AUTO⋅THEFT,B2,330,nan,2018-09-03⋅21:25:00,2018,9,Monday,21,Part⋅One,NORMANDY⋅ST,42.30607218,-71.0827326,"(42.30607218,⋅-71.08273260)"
I182070932,3301,Verbal⋅Disputes,VERBAL⋅DISPUTE,B2,584,nan,2018-09-03⋅20:39:37,2018,9,Monday,20,Part⋅Three,LAWN⋅ST,42.32701648,-71.10555088,"(42.32701648,⋅-71.10555088)"
I182070931,301,Robbery,ROBBERY⋅-⋅STREET,C6,177,nan,2018-09-03⋅20:48:00,2018,9,Monday,20,Part⋅One,MASSACHUSETTS⋅AVE,42.33152148,-71.07085307,"(42.33152148,⋅-71.07085307)"
I182070929,3301,Verbal⋅Disputes,VERBAL⋅DISPUTE,C11,364,nan,2018-09-03⋅20:38:00,2018,9,Monday,20,Part⋅Three,LESLIE⋅ST,42.29514664,-71.05860832,"(42.29514664,⋅-71.05860832)"


In [11]:
df.cols.dtypes("INCIDENT_NUMBER")

{'INCIDENT_NUMBER': 'object'}

In [8]:
df.head()

invalid escape sequence \d


,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


In [9]:
%%time
df["INCIDENT_NUMBER"].min().compute()

Wall time: 1.19 s


'142052550'

In [30]:
%%time
df.cols.min("*")

[('OFFENSE_CODE', {'min': dd.Scalar<series-..., dtype=int64>}), ('YEAR', {'min': dd.Scalar<series-..., dtype=int64>}), ('MONTH', {'min': dd.Scalar<series-..., dtype=int64>}), ('HOUR', {'min': dd.Scalar<series-..., dtype=int64>}), ('LAT', {'min': dd.Scalar<series-..., dtype=float64>}), ('LONG', {'min': dd.Scalar<series-..., dtype=float64>})]


distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.44 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.46 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.49 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.52 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 1.54 GB -- Worker memory limit: 2.00 GB
distributed.worker - WARN

Wall time: 5.78 s


{'OFFENSE_CODE': {'min': 111},
 'YEAR': {'min': 2015},
 'MONTH': {'min': 1},
 'HOUR': {'min': 0},
 'LAT': {'min': -1.0},
 'LONG': {'min': -71.17867378}}

In [99]:
!pip install graphviz

In [15]:
df.min().visualize()

ExecutableNotFound: failed to execute ['dot', '-Tpng'], make sure the Graphviz executables are on your systems' PATH

In [95]:
df.cols.create_exprs("*",df.functions.min)

[('OFFENSE_CODE', {'min': dd.Scalar<series-..., dtype=int64>}), ('YEAR', {'min': dd.Scalar<series-..., dtype=int64>}), ('MONTH', {'min': dd.Scalar<series-..., dtype=int64>}), ('HOUR', {'min': dd.Scalar<series-..., dtype=int64>}), ('Lat', {'min': dd.Scalar<series-..., dtype=float64>}), ('Long', {'min': dd.Scalar<series-..., dtype=float64>})]


[('OFFENSE_CODE', {'min': dd.Scalar<series-..., dtype=int64>}),
 ('YEAR', {'min': dd.Scalar<series-..., dtype=int64>}),
 ('MONTH', {'min': dd.Scalar<series-..., dtype=int64>}),
 ('HOUR', {'min': dd.Scalar<series-..., dtype=int64>}),
 ('Lat', {'min': dd.Scalar<series-..., dtype=float64>}),
 ('Long', {'min': dd.Scalar<series-..., dtype=float64>})]

In [82]:
df.cols.dtypes()

{'INCIDENT_NUMBER': 'object',
 'OFFENSE_CODE': 'int64',
 'OFFENSE_CODE_GROUP': 'object',
 'OFFENSE_DESCRIPTION': 'object',
 'DISTRICT': 'object',
 'REPORTING_AREA': 'object',
 'SHOOTING': 'object',
 'OCCURRED_ON_DATE': 'object',
 'YEAR': 'int64',
 'MONTH': 'int64',
 'DAY_OF_WEEK': 'object',
 'HOUR': 'int64',
 'UCR_PART': 'object',
 'STREET': 'object',
 'Lat': 'float64',
 'Long': 'float64',
 'Location': 'object'}

In [24]:
for c in df.cols.names():
    print(c)
    df.cols.min(c)

INCIDENT_NUMBER
OFFENSE_CODE
OFFENSE_CODE_GROUP
OFFENSE_DESCRIPTION
DISTRICT


TypeError: '<=' not supported between instances of 'str' and 'float'

In [90]:
import dask.array as da

In [96]:
df

In [100]:
da.histogram(df["INCIDENT_NUMBER"],bins=10, range=[9, 11])

AttributeError: 'tuple' object has no attribute 'compute'

In [ ]:
df.sum().compute()

In [14]:
%%time
from optimus.profiler.profiler import Profiler
p = Profiler()
p.run(df, "*")

TypeError: unsupported operand type(s) for ** or pow(): 'Array' and 'float'

In [16]:
df.ext.display(10)

INCIDENT_NUMBER 1 (object) not nullable,OFFENSE_CODE 2 (int64) not nullable,OFFENSE_CODE_GROUP 3 (object) not nullable,OFFENSE_DESCRIPTION 4 (object) not nullable,DISTRICT 5 (object) not nullable,REPORTING_AREA 6 (object) not nullable,SHOOTING 7 (object) not nullable,OCCURRED_ON_DATE 8 (object) not nullable,YEAR 9 (int64) not nullable,MONTH 10 (int64) not nullable,DAY_OF_WEEK 11 (object) not nullable,HOUR 12 (int64) not nullable,UCR_PART 13 (object) not nullable,STREET 14 (object) not nullable,Lat 15 (float64) not nullable,Long 16 (float64) not nullable,Location 17 (object) not nullable
I182070945,619,Larceny,LARCENY⋅ALL⋅OTHERS,D14,808,nan,2018-09-02⋅13:00:00,2018,9,Sunday,13,Part⋅One,LINCOLN⋅ST,42.35779134,-71.13937053,"(42.35779134,⋅-71.13937053)"
I182070943,1402,Vandalism,VANDALISM,C11,347,nan,2018-08-21⋅00:00:00,2018,8,Tuesday,0,Part⋅Two,HECLA⋅ST,42.30682138,-71.06030035,"(42.30682138,⋅-71.06030035)"
I182070941,3410,Towed,TOWED⋅MOTOR⋅VEHICLE,D4,151,nan,2018-09-03⋅19:27:00,2018,9,Monday,19,Part⋅Three,CAZENOVE⋅ST,42.34658879,-71.07242943,"(42.34658879,⋅-71.07242943)"
I182070940,3114,Investigate⋅Property,INVESTIGATE⋅PROPERTY,D4,272,nan,2018-09-03⋅21:16:00,2018,9,Monday,21,Part⋅Three,NEWCOMB⋅ST,42.33418175,-71.07866441,"(42.33418175,⋅-71.07866441)"
I182070938,3114,Investigate⋅Property,INVESTIGATE⋅PROPERTY,B3,421,nan,2018-09-03⋅21:05:00,2018,9,Monday,21,Part⋅Three,DELHI⋅ST,42.27536542,-71.09036101,"(42.27536542,⋅-71.09036101)"
I182070936,3820,Motor⋅Vehicle⋅Accident⋅Response,M/V⋅ACCIDENT⋅INVOLVING⋅PEDESTRIAN⋅-⋅INJURY,C11,398,nan,2018-09-03⋅21:09:00,2018,9,Monday,21,Part⋅Three,TALBOT⋅AVE,42.29019621,-71.07159012,"(42.29019621,⋅-71.07159012)"
I182070933,724,Auto⋅Theft,AUTO⋅THEFT,B2,330,nan,2018-09-03⋅21:25:00,2018,9,Monday,21,Part⋅One,NORMANDY⋅ST,42.30607218,-71.0827326,"(42.30607218,⋅-71.08273260)"
I182070932,3301,Verbal⋅Disputes,VERBAL⋅DISPUTE,B2,584,nan,2018-09-03⋅20:39:37,2018,9,Monday,20,Part⋅Three,LAWN⋅ST,42.32701648,-71.10555088,"(42.32701648,⋅-71.10555088)"
I182070931,301,Robbery,ROBBERY⋅-⋅STREET,C6,177,nan,2018-09-03⋅20:48:00,2018,9,Monday,20,Part⋅One,MASSACHUSETTS⋅AVE,42.33152148,-71.07085307,"(42.33152148,⋅-71.07085307)"
I182070929,3301,Verbal⋅Disputes,VERBAL⋅DISPUTE,C11,364,nan,2018-09-03⋅20:38:00,2018,9,Monday,20,Part⋅Three,LESLIE⋅ST,42.29514664,-71.05860832,"(42.29514664,⋅-71.05860832)"


In [ ]:
df.rows.to_list("OFFENSE_CODE")
# type(df["OFFENSE_CODE"])
# type(df.cols.select("OFFENSE_CODE"))

# a = [v for v in df["OFFENSE_CODE"].iteritems()]

# df[["OFFENSE_CODE"]].iteritems()


In [74]:
df.cols.select("OFFENSE_CODE")

,OFFENSE_CODE
npartitions=1,
,int64
,...


In [56]:
df.sample(1).head()

sample does not support the number of sampled items parameter, 'n'. Please use the 'frac' parameter instead.


,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
186823,I162085248,3006,Medical Assistance,SICK/INJURED/MEDICAL - PERSON,B2,314,NaN,2016-10-18 07:24:00,2016,10,Tuesday,7,Part Three,HOWLAND ST,42.314448,-71.089934,"(42.31444840, -71.08993418)"
65160,I182001199,1102,Fraud,FRAUD - FALSE PRETENSE / SCHEME,E18,526,NaN,2017-11-01 10:00:00,2017,11,Wednesday,10,Part Two,CLIFFORD ST,42.234543,-71.130915,"(42.23454320, -71.13091490)"
277221,I152095488,361,Robbery,ROBBERY - OTHER,D4,272,NaN,2015-11-17 14:52:00,2015,11,Tuesday,14,Part One,WASHINGTON ST,42.334831,-71.079041,"(42.33483110, -71.07904134)"
44193,I182023593,3805,Motor Vehicle Accident Response,M/V ACCIDENT - POLICE VEHICLE,D4,171,NaN,2018-03-30 09:49:00,2018,3,Friday,9,Part Three,ALBANY ST,42.334288,-71.072395,"(42.33428841, -71.07239518)"
13308,I182056371,3115,Investigate Person,INVESTIGATE PERSON,C11,250,NaN,2018-07-17 20:18:00,2018,7,Tuesday,20,Part Three,SAVIN HILL AVE,42.312482,-71.048290,"(42.31248165, -71.04829028)"


In [29]:
df.cols.schema_dtype()

[numpy.object_,
 numpy.int64,
 numpy.object_,
 numpy.object_,
 numpy.object_,
 numpy.object_,
 numpy.object_,
 numpy.object_,
 numpy.int64,
 numpy.int64,
 numpy.object_,
 numpy.int64,
 numpy.object_,
 numpy.object_,
 numpy.float64,
 numpy.float64,
 numpy.object_]

In [28]:
import numpy as np
np.dtype(df["OFFENSE_CODE"]).type

numpy.int64

In [77]:
df.schema

AttributeError: 'DataFrame' object has no attribute 'schema'

In [7]:
%%time
df["INCIDENT_NUMBER"].value_counts().nlargest(5).compute()

Wall time: 2.51 s


I162030584    13
I152080623    11
I172013170    10
I182065208    10
I172096394    10
Name: INCIDENT_NUMBER, dtype: int64

In [14]:
df["INCIDENT_NUMBER"].nunique().compute()

282517

In [6]:
df.npartitions

1

In [7]:
df = df.repartition(npartitions=20)

In [8]:
df.npartitions

20

In [73]:
%%time
df["INCIDENT_NUMBER"].value_counts().nlargest(5).compute()

Wall time: 8.07 s


I162030584    13
I152080623    11
I172013170    10
I182065208    10
I172096394    10
Name: INCIDENT_NUMBER, dtype: int64

In [16]:
df["INCIDENT_NUMBER"].unique().count().compute()

282517

In [6]:
df.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


In [87]:
import dask.dataframe as dd

In [7]:
def count_uniques_agg(col_name, args=True):
    estimate = args[0]
    def count_uniques_agg_(x):
        print(estimate)
        if estimate is True:
            result = {"count_uniques_agg": x[col_name].nunique_approx()}
        else:
            result = {"count_uniques_agg": x[col_name].nunique()}
        return result
    return count_uniques_agg_

In [98]:
df.rows.count()

319073

In [39]:
import dask.array as da

In [224]:
def hist_agg(col_name, args):
    df = args[0]
    bins = args[1]
    range = args[2]
    
    def hist_agg_(x):
        return {"hist_agg":list(da.histogram(x[col_name], bins=bins, range=range)[1]) }
    return hist_agg_

In [254]:
def percentile_agg(col_name, args):
    values = args[0]

    def _percentile(x):

        return {"percentile_agg": {i:j for i,j in x[col_name].quantile(values).iteritems()}}

    return _percentile

In [259]:
%%time
import dask.dataframe as dd
# k, s, m = [stats.kurtosis(x), stats.skew(x), stats.moment(x, 5)]
# agg_func=["min","max"]
# agg_func=[percentile_agg(0.1)(df),"min","max", count_uniques_agg("INCIDENT_NUMBER",False)(df)]
# agg_func=[percentile_agg(0.1)(df),"min","max"]
agg_func=[hist_agg]

# agg_func=[df.functions.count_na_agg]

exprs = df.cols.create_exprs(["OFFENSE_CODE","YEAR"], agg_func, df, 10,[1,5])

# print(exprs)
print(dd.compute(exprs))

([('OFFENSE_CODE', {'hist_agg': [1.0, 1.4, 1.8, 2.2, 2.6, 3.0, 3.4000000000000004, 3.8000000000000003, 4.2, 4.6, 5.0]}), ('YEAR', {'hist_agg': [1.0, 1.4, 1.8, 2.2, 2.6, 3.0, 3.4000000000000004, 3.8000000000000003, 4.2, 4.6, 5.0]})],)
Wall time: 4 ms


dask.dataframe.core.DataFrame

In [221]:
list(r["OFFENSE_CODE"]["hist_agg"][1])

[4.5, 4.6, 4.7, 4.8, 4.9, 5.0, 5.1, 5.2, 5.3, 5.4, 5.5]

In [33]:
for i in x:
    print(i[0],i[1])

a {'min': 111}


In [124]:
print(df.cols.exec_agg(exprs))

{'OFFENSE_CODE': {'kurtosis': 1.5838041686767816, 'skew': array(-0.40994654), 'zeros': 15106, 'variance': 1404901.8183066112}, 'HOUR': {'kurtosis': 2.3994270670757927, 'skew': array(-0.4834518), 'zeros': 15106, 'variance': 39.61701992983649}}


In [161]:
import dask.dataframe as dd

In [159]:
df.min()

Dask Series Structure:
npartitions=1
DAY_OF_WEEK    object
YEAR              ...
dtype: object
Dask Name: dataframe-min-agg, 5 tasks

In [158]:
df.functions.percentile_agg(0.1)(df)

Dask Series Structure:
npartitions=1
HOUR    float64
YEAR        ...
Name: 0.1, dtype: float64
Dask Name: quantiles-concat, 28 tasks

In [150]:
dd.compute(df.min())

(INCIDENT_NUMBER                                           142052550
 OFFENSE_CODE                                                    111
 OFFENSE_CODE_GROUP                               Aggravated Assault
 OFFENSE_DESCRIPTION    A&B HANDS, FEET, ETC.  - MED. ATTENTION REQ.
 REPORTING_AREA                                                     
 OCCURRED_ON_DATE                                2015-06-15 00:00:00
 YEAR                                                           2015
 MONTH                                                             1
 DAY_OF_WEEK                                                  Friday
 HOUR                                                              0
 Lat                                                              -1
 Long                                                       -71.1787
 Location                                 (-1.00000000, -1.00000000)
 dtype: object,)

In [13]:
import dask.dataframe as dd

In [74]:
def frequency(col_name):
    def frequency_(x):
        print(x)
        return 
    return frequency_

In [79]:
df.cols.frequency("INCIDENT_NUMBER")

({'I162030584': 13,
  'I152080623': 11,
  'I172013170': 10,
  'I182065208': 10,
  'I172096394': 10},)

In [76]:
print(dd.compute(df[col_name].value_counts().nlargest(5)))

(<function frequency.<locals>.frequency_ at 0x00000202489E19D8>,)


In [17]:
%%time
dd.compute(df["OFFENSE_CODE"].min(), df["INCIDENT_NUMBER"].value_counts().nlargest(5), df["OFFENSE_CODE"].value_counts().nlargest(5), df["OFFENSE_CODE"].nunique())

Wall time: 2.01 s


(111, I162030584    13
 I152080623    11
 I172013170    10
 I182065208    10
 I172096394    10
 Name: INCIDENT_NUMBER, dtype: int64, 3006    18783
 3115    18754
 3831    16323
 1402    15154
 802     14799
 Name: OFFENSE_CODE, dtype: int64, 222)

In [118]:
%%time
dd.compute(df["OFFENSE_CODE"].nunique())

Wall time: 1.26 s


(222,)

In [27]:
## https://distributed.dask.org/en/latest/web.html

NameError: name 'cluster' is not defined

In [56]:
# Another approach
# from dask.array import stats
# x = da.random.beta(1, 1, size=(1000,), chunks=10)
# k, s, m = [stats.kurtosis(x), stats.skew(x), stats.moment(x, 5)]
# dask.compute(k, s, m)

(1.7612340817172787, -0.064073498030693302, -0.00054523780628304799)

(1.7612340817172787, -0.0640734980306933, -0.000545237806283048)

In [288]:
import pandas as pd
data = [['tom', 10], ['nick', 15], ['juli', 0],['argenis', 10]]

# Create the pandas DataFrame 
df = pd.DataFrame(data, columns = ['Name', 'Age']) 
df.cols.count_by_dtypes("*")

ddf = dd.from_pandas(df, npartitions=2)

In [9]:
df.reset_index(drop=True)

,Name,Age
0,tom,10
1,nick,15
2,juli,0
3,argenis,10


In [20]:
# df[df == 0].count(axis=0)
# Speedy https://stackoverflow.com/questions/35277075/python-pandas-counting-the-occurrences-of-a-specific-value
def zeros(series):
    return (series.values==0).sum()

zeros(df["Age"])

1

In [21]:
def percentile_agg(n):
    def percentile_(x):
        return x.quantile(n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

# percentile_agg(values=[0.1, 0.5])

In [28]:
print(ddf)

Dask DataFrame Structure:
                 Name    Age
npartitions=2               
0              object  int64
2                 ...    ...
3                 ...    ...
Dask Name: from_pandas, 2 tasks


In [56]:
dd.compute(ddf.min(), ddf.max())

(Name    argenis
 Age           0
 dtype: object, Name    tom
 Age      15
 dtype: object)

In [103]:
ddf.agg(
    {
#         # find the min, max, and sum of the duration column
#         'Name': [min, max, sum],
#          # find the number of network type entries
#         'Age': "count",
        # min, first, and number of unique dates per group
        'Age': [min, 'max', 'nunique', zeros , percentile_agg(1)]
    }
).to_dict()

AttributeError: 'DataFrame' object has no attribute 'agg'

In [351]:
df

,Name,Age
0,tom,10
1,nick,15
2,juli,0
3,argenis,10


In [266]:
def agg_fn(x):
    result = pd.Series(
        dict(
            C = x.quantile(1.0), # int           
        )
    )
    print(result)
    return result

In [267]:
a=df[['Age']]

In [268]:
a.apply(agg_fn)

C    15.0
dtype: float64
C    15.0
dtype: float64


,Age
C,15.0


In [284]:
a.apply(agg_fn)

C    15.0
dtype: float64
C    15.0
dtype: float64


,Age
C,15.0


In [269]:
print(df)

      Name  Age
0      tom   10
1     nick   15
2     juli    0
3  argenis   10


In [ ]:

#https://stackoverflow.com/questions/46080171/constructing-mode-and-corresponding-count-functions-using-custom-aggregation-fun

In [31]:


def chunk(s):
    """
    The function applied to the
    individual partition (map)
    """
    return s.apply(lambda x: list(set(x)))

def agg(s):
    """
    The function whic will aggrgate
    the result from all the partitions(reduce)
    """
    s = s._selected_obj
    return s.groupby(level=list(range(s.index.nlevels))).sum()

def finalize(s):
    """
    The optional functional that will be
    applied to the result of the agg_tu functions
    """
    return s.apply(lambda x: len(set(x)))

tunique = dd.Aggregation('tunique', chunk, agg, finalize)

In [42]:
df.groupby(['INCIDENT_NUMBER']).agg({'INCIDENT_NUMBER': tunique}).compute()

,INCIDENT_NUMBER
INCIDENT_NUMBER,
142052550,1
I010370257-00,1
I030217815-08,1
I050310906-00,1
I060168073-00,1
I080542626-00,1
I090317057-00,1
I090321958-00,1
I100033064-00,1


distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://192.168.0.9/34804/10 remote=inproc://192.168.0.9/34804/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://192.168.0.9/34804/11 remote=inproc://192.168.0.9/34804/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://192.168.0.9/34804/12 remote=inproc://192.168.0.9/34804/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://192.168.0.9/34804/13 remote=inproc://192.168.0.9/34804/1>
distributed.comm.inproc - WARNING - Closing dangling queue in <InProc local=inproc://192.168.0.9/34804/14 remote=inproc://192.168.0.9/34804/1>


In [109]:
df.apply({'g0': unique}, axis=1).compute()

NameError: name 'unique' is not defined

In [15]:
from abc import abstractmethod, ABC


class AbstractCols(ABC):    
    @abstractmethod
    def min(self, columns):
        pass

In [16]:
class BaseCols(AbstractCols):
    def __init__(self, functions):
        self.functions = functions
                
    @staticmethod
    # @abstractmethod
    def get_agg_function():
        return 3
    
    def min(self, columns):
        return BaseCols.agg_exprs(columns, self.functions)
    
    @staticmethod
    def agg_exprs(columns, agg):
        return agg

In [21]:
functions={"hola1"}

class DaskCols(BaseCols):
    def __init__(self):
        super().__init__(functions)
    
c = DaskCols()

In [22]:
print(c.min("cols"))

{'hola1'}
